In [46]:
import numpy as np
import ufl
from mpi4py import MPI
from dolfinx import mesh, fem, nls
from petsc4py import PETSc
from dolfinx.fem import functionspace

L = 100.0        # domain length
nx = 400         # number of elements
domain = mesh.create_interval(MPI.COMM_WORLD, nx, [0.0, L])

V = V = functionspace(domain, ("Lagrange", 1))



rho_val = 1.0   # density
mu_val = 1.0    # shear modulus

rho = fem.Constant(domain, PETSc.ScalarType(rho_val))
mu = fem.Constant(domain, PETSc.ScalarType(mu_val))

dt = 0.01       # time step
tmax = 2.0      # final time
nt = int(tmax/dt)

# Previous, current, next displacements
u_n = fem.Function(V)
u = fem.Function(V)
v = fem.Function(V)  # velocity


dt = 0.01       # time step
tmax = 2.0      # final time
nt = int(tmax/dt)

# Previous, current, next displacements
u_n = fem.Function(V)
u = fem.Function(V)
v = fem.Function(V)  # velocity


u_trial = ufl.TrialFunction(V)
v_test = ufl.TestFunction(V)

# Mass and stiffness forms (semi-discrete)
a_mass = u_trial*v_test*ufl.dx

a_stiff = mu * ufl.inner(ufl.grad(u_trial), ufl.grad(v_test)) * ufl.dx

# Assemble matrices
M = fem.petsc.assemble_matrix(fem.form(a_mass))
M.assemble()
K = fem.petsc.assemble_matrix(fem.form(a_stiff))
K.assemble()

def ricker(t, f0=10.0):
    return (1-2*(np.pi*f0*(t-1/f0))**2)*np.exp(-(np.pi*f0*(t-1/f0))**2)

x = domain.geometry.x
f = fem.Function(V)
f.x.array[:] = 0.0
f.x.array[0] = ricker(0.0)  # initial


# Convert to PETSc vectors
u_n_vec = u_n.x
u_vec = u.x
v_vec = v.x

# Create PETSc solver for M
solver = PETSc.KSP().create(MPI.COMM_WORLD)
solver.setOperators(M)
solver.setType(PETSc.KSP.Type.PREONLY)            # direct solver
solver.getPC().setType(PETSc.PC.Type.LU)          # LU factorization
solver.setFromOptions()

for n_step in range(nt):
    t = n_step * dt
    f.x.array[0] = ricker(t)

    # Assemble source vector
    rhs = fem.assemble_vector(fem.form(f*v_test*ufl.dx))

    # Subtract K*u_n
    K.mult(u_n.x, tmp_vec)
    rhs.axpy(-1.0, tmp_vec)

    # Add M*u_n
    M.mult(u_n.x, tmp_vec)
    rhs.axpy(1.0, tmp_vec)

    # Scale by dt^2
    rhs.scale(dt**2)

    # Solve M*u_next = rhs
    solver.solve(rhs, u.x)

    # Update previous step
    u_n.x.array[:] = u.x.array[:]

NameError: name 'tmp_vec' is not defined

In [23]:
fem.FunctionSpace?

Init signature:
fem.FunctionSpace(
    mesh: 'Mesh',
    element: 'ufl.FiniteElementBase',
    cppV: 'typing.Union[_cpp.fem.FunctionSpace_float32, _cpp.fem.FunctionSpace_float64]',
)
Docstring:      A space on which Functions (fields) can be defined.
Init docstring:
Create a finite element function space.

Note:
    This initialiser is for internal use and not normally called
    in user code. Use :func:`functionspace` to create a function
    space.

Args:
    mesh: Mesh that space is defined on.
    element: UFL finite element.
    cppV: Compiled C++ function space.
File:           ~/miniforge3/envs/fenicsx-env/lib/python3.13/site-packages/dolfinx/fem/function.py
Type:           ABCMeta
Subclasses:     